In [1]:
import pandas as pd
import numpy as np
import os
import datetime
from google.cloud import bigquery

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# 1. Cargar DF

Cargamos los tres archivos y los inspeccionamos.

In [3]:
from google.colab import auth
auth.authenticate_user()

client = bigquery.Client(project='freshly-case-study-468308')


query_products = "SELECT * FROM freshly_data.INFORMACION_PRODUCTOS"
df_products = client.query(query_products).to_dataframe()

query_sells = "SELECT * FROM freshly_data.venta_sku_2024"
df_sells = client.query(query_sells).to_dataframe()

query_orders = "SELECT * FROM freshly_data.INFORMACION_PEDIDOS"
df_orders = client.query(query_orders).to_dataframe()

Inspeccionamos los DataFrames

In [4]:
df_products.head()

,SKU,BC_descripcion,activo_estudio,componente_pack,FF_CODE,descripcion_familia,tipo_producto,pk_id_categoria,id_categoria,categoria,descripcion_categoria,id_subcategoria,subcategoria,descripcion_subcategoria,tipo_uso,zona_aplicacion,num_paso_rutina,paso_rutina,id_formato_size,id_formato,size,unidad_medida,colaboracion,id_presentacion,presentacion,id_envase,material_envase,id_color,color,segmento_kids,segmento_menopausia,apto_embarazo,concern_00,concern_01,concern_02,concern_03,concern_04,concern_05,concern_06,concern_07,concern_08,concern_09,concern_10,concern_11,concern_12,concern_13,concern_14,concern_15,concern_16,concern_17,concern_18,concern_19,concern_20,concern_21
0,M990237,Calendario adviento 2024,DES,False,FF-095,PACK,Pack,None,None,None,None,None,None,None,None,None,<NA>,None,None,None,NaN,None,False,P1,Pack,M6,Otro,None,None,False,False,SI,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,M021272,(OLD) Productos INT - Mini Glow Up Body Pack,DES,False,FF-095,PACK,Pack,None,None,None,None,None,None,None,None,None,<NA>,None,None,None,NaN,None,False,P1,Pack,M6,Otro,None,None,False,False,SI,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,M021077,(OLD) Productos INT - Must Have Pack,DES,False,FF-095,PACK,Pack,None,None,None,None,None,None,None,None,None,<NA>,None,None,None,NaN,None,False,P1,Pack,M6,Otro,None,None,False,False,SI,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,M021072,Productos INT - Glow Foundation & Concealer Pack,DES,False,FF-095,PACK,Pack,None,None,None,None,None,None,None,None,None,<NA>,None,None,None,NaN,None,False,P1,Pack,M6,Otro,None,None,False,False,SI,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,M021300,Productos INT - Infinite Wonder Pack,DES,False,FF-095,PACK,Pack,None,None,None,None,None,None,None,None,None,<NA>,None,None,None,NaN,None,False,P1,Pack,M6,Otro,None,None,False,False,SI,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [5]:
df_sells.head()

,sku,albaran,unidades_vendidas,importe_sin_iva,clientenuevo_repetidor,ranking_pedido_DESC,gifting
0,M020901,ALV24-1068969,8,0.00,Cliente Repetidor,122240,1
1,M020908,ALV24-564168,1,25.67,Cliente Repetidor,4,0
2,M020908,ALV24-663193,1,0.00,Cliente Repetidor,168111,1
3,M020908,ALV24-770569,1,29.68,Cliente Repetidor,4,0
4,M020908,ALV24-773129,1,22.39,Cliente Repetidor,8,0


In [6]:
df_orders.head()

,tipo_venta,BC_num_cliente,mercado,fecha_pedido,albaran,fecha_albaran
0,Pedido B2B,C0433338,FR,2024-12-11 00:00:00+00:00,ALV24-1081058,2024-12-11
1,Pedido B2B,C1915287,PT,2024-06-21 00:00:00+00:00,ALV24-478144,2024-06-21
2,Pedido B2B,C1836605,FR,2024-06-25 00:00:00+00:00,ALV24-483040,2024-06-25
3,Pedido B2B,C2090043,IT,2024-08-05 00:00:00+00:00,ALV24-700851,2024-08-05
4,Pedido B2B,C1915287,PT,2024-11-13 00:00:00+00:00,ALV24-894603,2024-11-13


## 1.1 Unión DataFrames

Para poder saber qué productos se han vendido en `df_sells`, necesitamos relacionar su columna `sku` con la columna `SKU` de `df_products`.

Por ello, unimos ambos DataFrames haciendo coincidir dichas columnas y conservando la columna `sku` de `df_sells`, de forma que mantenemos todas las ventas aunque no tengamos información de producto en `df_products`.

In [7]:
df = df_sells.merge(df_products, left_on='sku', right_on='SKU', how='left')

In [8]:
# Verificamos
df.head()

,sku,albaran,unidades_vendidas,importe_sin_iva,clientenuevo_repetidor,ranking_pedido_DESC,gifting,SKU,BC_descripcion,activo_estudio,componente_pack,FF_CODE,descripcion_familia,tipo_producto,pk_id_categoria,id_categoria,categoria,descripcion_categoria,id_subcategoria,subcategoria,descripcion_subcategoria,tipo_uso,zona_aplicacion,num_paso_rutina,paso_rutina,id_formato_size,id_formato,size,unidad_medida,colaboracion,id_presentacion,presentacion,id_envase,material_envase,id_color,color,segmento_kids,segmento_menopausia,apto_embarazo,concern_00,concern_01,concern_02,concern_03,concern_04,concern_05,concern_06,concern_07,concern_08,concern_09,concern_10,concern_11,concern_12,concern_13,concern_14,concern_15,concern_16,concern_17,concern_18,concern_19,concern_20,concern_21
0,M020901,ALV24-1068969,8,0.00,Cliente Repetidor,122240,1,M020901,Prod. PT - Peony Limitless Eyelash Serum - 5ml,ACT,True,FF-112,SERUM PESTAÑAS,Serum,CAT-C1-003,C1,Cuidado Facial,"Productos diseñados para el rostro, incluyendo...",C1-003,Sérums Faciales,Concentrados con activos específicos para trat...,Cuidado,Facial,3,Paso 3: Facial,LIQ-05,LIQ,5.0,ml,False,P2,Full Size,M1,Vidrio,None,None,False,False,SI,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False
1,M020908,ALV24-564168,1,25.67,Cliente Repetidor,4,0,M020908,Prod. PT - Hyaluronic Energy Body Serum - 200ml,ACT,True,FF-072,HYALURONIC,Gel,CAT-C2-004,C2,Cuidado Corporal,"Productos enfocados en la hidratación, limpiez...",C2-004,Tratamientos Corporales,"Fórmulas con activos específicos para firmeza,...",Cuidado,Corporal,4,Paso 4: Corporal,LIQ-12,LIQ,200.0,ml,False,P2,Full Size,M1,Vidrio,None,None,False,False,SI,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,True,False,False
2,M020908,ALV24-663193,1,0.00,Cliente Repetidor,168111,1,M020908,Prod. PT - Hyaluronic Energy Body Serum - 200ml,ACT,True,FF-072,HYALURONIC,Gel,CAT-C2-004,C2,Cuidado Corporal,"Productos enfocados en la hidratación, limpiez...",C2-004,Tratamientos Corporales,"Fórmulas con activos específicos para firmeza,...",Cuidado,Corporal,4,Paso 4: Corporal,LIQ-12,LIQ,200.0,ml,False,P2,Full Size,M1,Vidrio,None,None,False,False,SI,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,True,False,False
3,M020908,ALV24-770569,1,29.68,Cliente Repetidor,4,0,M020908,Prod. PT - Hyaluronic Energy Body Serum - 200ml,ACT,True,FF-072,HYALURONIC,Gel,CAT-C2-004,C2,Cuidado Corporal,"Productos enfocados en la hidratación, limpiez...",C2-004,Tratamientos Corporales,"Fórmulas con activos específicos para firmeza,...",Cuidado,Corporal,4,Paso 4: Corporal,LIQ-12,LIQ,200.0,ml,False,P2,Full Size,M1,Vidrio,None,None,False,False,SI,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,True,False,False
4,M020908,ALV24-773129,1,22.39,Cliente Repetidor,8,0,M020908,Prod. PT - Hyaluronic Energy Body Serum - 200ml,ACT,True,FF-072,HYALURONIC,Gel,CAT-C2-004,C2,Cuidado Corporal,"Productos enfocados en la hidratación, limpiez...",C2-004,Tratamientos Corporales,"Fórmulas con activos específicos para firmeza,...",Cuidado,Corporal,4,Paso 4: Corporal,LIQ-12,LIQ,200.0,ml,False,P2,Full Size,M1,Vidrio,None,None,False,False,SI,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,True,False,False


Ahora unimos el DataFrame `df` con `df_orders` para añadir información sobre los pedidos asociado a cada albaran.

In [9]:
df = df.merge(df_orders, on='albaran', how='left')

In [10]:
# Verificamos
df.head()

,sku,albaran,unidades_vendidas,importe_sin_iva,clientenuevo_repetidor,ranking_pedido_DESC,gifting,SKU,BC_descripcion,activo_estudio,componente_pack,FF_CODE,descripcion_familia,tipo_producto,pk_id_categoria,id_categoria,categoria,descripcion_categoria,id_subcategoria,subcategoria,descripcion_subcategoria,tipo_uso,zona_aplicacion,num_paso_rutina,paso_rutina,id_formato_size,id_formato,size,unidad_medida,colaboracion,id_presentacion,presentacion,id_envase,material_envase,id_color,color,segmento_kids,segmento_menopausia,apto_embarazo,concern_00,concern_01,concern_02,concern_03,concern_04,concern_05,concern_06,concern_07,concern_08,concern_09,concern_10,concern_11,concern_12,concern_13,concern_14,concern_15,concern_16,concern_17,concern_18,concern_19,concern_20,concern_21,tipo_venta,BC_num_cliente,mercado,fecha_pedido,fecha_albaran
0,M020901,ALV24-1068969,8,0.00,Cliente Repetidor,122240,1,M020901,Prod. PT - Peony Limitless Eyelash Serum - 5ml,ACT,True,FF-112,SERUM PESTAÑAS,Serum,CAT-C1-003,C1,Cuidado Facial,"Productos diseñados para el rostro, incluyendo...",C1-003,Sérums Faciales,Concentrados con activos específicos para trat...,Cuidado,Facial,3,Paso 3: Facial,LIQ-05,LIQ,5.0,ml,False,P2,Full Size,M1,Vidrio,None,None,False,False,SI,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,Pedido B2B,C2198402,ES,2024-12-03 00:00:00+00:00,2024-12-03
1,M020908,ALV24-564168,1,25.67,Cliente Repetidor,4,0,M020908,Prod. PT - Hyaluronic Energy Body Serum - 200ml,ACT,True,FF-072,HYALURONIC,Gel,CAT-C2-004,C2,Cuidado Corporal,"Productos enfocados en la hidratación, limpiez...",C2-004,Tratamientos Corporales,"Fórmulas con activos específicos para firmeza,...",Cuidado,Corporal,4,Paso 4: Corporal,LIQ-12,LIQ,200.0,ml,False,P2,Full Size,M1,Vidrio,None,None,False,False,SI,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,True,False,False,Pedido Ecommerce,C1969891,IT,2024-07-01 23:46:58+00:00,2024-07-04
2,M020908,ALV24-663193,1,0.00,Cliente Repetidor,168111,1,M020908,Prod. PT - Hyaluronic Energy Body Serum - 200ml,ACT,True,FF-072,HYALURONIC,Gel,CAT-C2-004,C2,Cuidado Corporal,"Productos enfocados en la hidratación, limpiez...",C2-004,Tratamientos Corporales,"Fórmulas con activos específicos para firmeza,...",Cuidado,Corporal,4,Paso 4: Corporal,LIQ-12,LIQ,200.0,ml,False,P2,Full Size,M1,Vidrio,None,None,False,False,SI,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,True,False,False,Pedido B2B,C1547947,ES,2024-07-18 00:00:00+00:00,2024-07-18
3,M020908,ALV24-770569,1,29.68,Cliente Repetidor,4,0,M020908,Prod. PT - Hyaluronic Energy Body Serum - 200ml,ACT,True,FF-072,HYALURONIC,Gel,CAT-C2-004,C2,Cuidado Corporal,"Productos enfocados en la hidratación, limpiez...",C2-004,Tratamientos Corporales,"Fórmulas con activos específicos para firmeza,...",Cuidado,Corporal,4,Paso 4: Corporal,LIQ-12,LIQ,200.0,ml,False,P2,Full Size,M1,Vidrio,None,None,False,False,SI,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,True,False,False,Pedido Ecommerce,C1953271,IT,2024-09-08 09:49:59+00:00,2024-09-09
4,M020908,ALV24-773129,1,22.39,Cliente Repetidor,8,0,M020908,Prod. PT - Hyaluronic Energy Body Serum - 200ml,ACT,True,FF-072,HYALURONIC,Gel,CAT-C2-004,C2,Cuidado Corporal,"Productos enfocados en la hidratación, limpiez...",C2-004,Tratamientos Corporales,"Fórmulas con activos específicos para firmeza,...",Cuidado,Corporal,4,Paso 4: Corporal,LIQ-12,LIQ,200.0,ml,False,P2,Full Size,M1,Vidrio,None,None,False,False,SI,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,True,False,False,Pedido Ecommerce,C0129189,ES,2024-09-08 21:57:52+00:00,2024-09-09


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3366991 entries, 0 to 3366990
Data columns (total 66 columns):
 #   Column                    Dtype              
---  ------                    -----              
 0   sku                       object             
 1   albaran                   object             
 2   unidades_vendidas         Int64              
 3   importe_sin_iva           float64            
 4   clientenuevo_repetidor    object             
 5   ranking_pedido_DESC       Int64              
 6   gifting                   Int64              
 7   SKU                       object             
 8   BC_descripcion            object             
 9   activo_estudio            object             
 10  componente_pack           boolean            
 11  FF_CODE                   object             
 12  descripcion_familia       object             
 13  tipo_producto             object             
 14  pk_id_categoria           object             
 15  id_categoria   

# 2. Manipulación de datos

In [12]:
# borramos columna repetida `SKU``
df.drop(columns='SKU', inplace=True)

## 2.1 Lidiar con los NaN

In [13]:
# Identificamos los valores NaN
df.isna().sum()

,0
sku,0
albaran,0
unidades_vendidas,0
importe_sin_iva,0
clientenuevo_repetidor,0
ranking_pedido_DESC,16780
gifting,0
BC_descripcion,1621
activo_estudio,1621
componente_pack,1621


Al ser un dataset muy grande, con más de 3 millones de filas, me puedo permitir tratar los valores nulos (`NaN`) eliminando directamente las filas afectadas en muchos casos. Por ejemplo, las columnas con 2.440 valores nulos solo representan un 0,07% del total, un impacto despreciable.

Voy a tratar los `NaN` de la siguiente manera:

 Eliminación de filas con `NaN`


- `ranking_pedido_DESC`: tiene 16.780 filas nulas → **eliminar esas filas**.
- `segmento_kids` y otras columnas con 1.621 valores nulos → **eliminar las filas**.
- `categoria` y otras columnas con 68.330 valores nulos → **eliminar las filas**.
- `BC_num_cliente` y otras columnas con 16.780 valores nulos → **eliminar las filas**.

Imputación o sustitución

- `num_paso_rutina`, `paso_rutina`, `id_formato`, `id_formato_size`, `zona_aplicacion`, `id_presentacion`, `presentacion`, `id_presentacion` y `material_envase` → reemplazar `NaN` por `"Unknown"`.
- `size` y `num_paso_rutina` → reemplazar `NaN` por 0 para mantener formato

Eliminación de columnas

- `id_color` y `color` → **eliminar las columnas**, ya que falta demasiada información relevante.

Tratamiento especial

- `fecha_pedido` y `fecha_albaran` → mantener los valores como `NaT` (`Not a Time`) para compatibilidad con análisis temporal o tratamiento posterior.

In [14]:
# Eliminar filas con NaN en 'albaran', lo que a su vez elimina otras columnas relacionadas (16.780 filas)
df = df[df['albaran'].notna()]

# Lo mismo con segmento_kids y 1.621 filas
df = df.dropna(subset='segmento_kids')

# Lo mismo con categoria y 68.330 filas
df = df.dropna(subset='categoria')

# Lo mismo con BC_num_cliente y 16.780 filas
df = df.dropna(subset='BC_num_cliente')

In [15]:
# Sustituimos los NaN a Unknown
cols_to_fill = [
    'paso_rutina',
    'zona_aplicacion',
    'presentacion',
    'material_envase',
    'tipo_venta',
    'id_formato_size',
    'id_formato',
    'unidad_medida',
    'id_envase',
    'id_presentacion'

]

df[cols_to_fill] = df[cols_to_fill].fillna('Unknown')

In [16]:
# Eliminamos columnas color y id_color por falta de informacion
df = df.drop(columns=['color', 'id_color'])

In [17]:
# Sustituimos NaN en la columna size por 0 para mantener el formato float
df['size'] = df['size'].fillna(0)
df['num_paso_rutina'] = df['num_paso_rutina'].fillna(0)

Finalmente verificamos que no queden valores NaN

In [18]:
df.isna().any()

,0
sku,False
albaran,False
unidades_vendidas,False
importe_sin_iva,False
clientenuevo_repetidor,False
ranking_pedido_DESC,False
gifting,False
BC_descripcion,False
activo_estudio,False
componente_pack,False


In [19]:
df.shape

(3281891, 63)

El DataFrame contiene 3.281.891 filas después de la limpieza y manipulación de los valores NaN. Partiendo de 3.366.991 filas en el DataFrame original, se ha eliminado aproximadamente un 2,53% de las filas.

In [20]:
df = df.copy()

## 2.2 Transformaciones de tipos y limpieza previa

Realizaré los siguientes cambios para mejor manipulacion del DataFrame:

- `gifting` → Convertir a Boolean
- `apto_embarazo` → Convertir a Boolean
- `importe_sin_iva` → Imputar un precio estimado en las filas donde `gifting` = True, utilizando la mediana del precio_unitario` por SKU, con el` fin de permitir un análisis posterior del valor económico regalado.
- `concern_xx` → Crear un nuevo DataFrame con la suma de cada `concern` para analizar las preocupaciones más frecuentes de los clientes.
- Duplicados → Comprobar que no existan combinaciones duplicadas de `albaran` y `sku`, asegurando que cada producto aparece solo una vez por pedido.
- `fecha_albaran` → Convertir a formato DateTime

### 2.2.1 Convertimos las columnas a tipo booleano

In [21]:
df['gifting'] = df['gifting'].astype(bool)

In [22]:
df['apto_embarazo'] = df['apto_embarazo'].map({'SI': True, 'NO': False})

### 2.2.2 Crear un nuevo DataFrame con la suma de cada `concern` para analizar las preocupaciones más frecuentes de los clientes.

In [24]:
# Lista de concern
concerns_cols = [col for col in df.columns if col.startswith('concern_')]

# Columnas que conservare
cols_keep = ['fecha_albaran', 'sku', 'BC_num_cliente', 'categoria', 'mercado'] + concerns_cols

# Filtrar filas con concern True
df_concerns = df[cols_keep].copy()
df_concerns = df_concerns[df_concerns[concerns_cols].sum(axis=1) > 0]

# Pivotar la tabla
concerns_long = df_concerns.melt(
    id_vars=['fecha_albaran', 'sku', 'BC_num_cliente', 'categoria', 'mercado'],
    value_vars=concerns_cols,
    var_name='Concern',
    value_name='Value'
)

# Filtrar solo los True
df_concerns = concerns_long[concerns_long['Value'] == True].drop(columns='Value')

### 2.2.3 Identificar duplicados

In [25]:
duplicados = df.duplicated(subset=['albaran', 'sku'])
df_duplicados = df[duplicados]

print(f"Filas duplicadas: {duplicados.sum()}")

Filas duplicadas: 41375


Se eliminan las filas duplicadas que tienen la misma combinación de `albaran` y `sku`, manteniendo solo la primera ocurrencia. Esto asegura que no haya líneas repetidas para un mismo producto dentro de un mismo albarán.

In [26]:
df = df.drop_duplicates(subset=['albaran', 'sku'], keep='first')

### 2.2.4 fecha_albaran a DateTime

In [27]:
df['fecha_albaran'] = pd.to_datetime(df['fecha_albaran'], errors='coerce')

In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3240516 entries, 0 to 3366990
Data columns (total 64 columns):
 #   Column                    Dtype              
---  ------                    -----              
 0   sku                       object             
 1   albaran                   object             
 2   unidades_vendidas         Int64              
 3   importe_sin_iva           float64            
 4   clientenuevo_repetidor    object             
 5   ranking_pedido_DESC       Int64              
 6   gifting                   bool               
 7   BC_descripcion            object             
 8   activo_estudio            object             
 9   componente_pack           boolean            
 10  FF_CODE                   object             
 11  descripcion_familia       object             
 12  tipo_producto             object             
 13  pk_id_categoria           object             
 14  id_categoria              object             
 15  categoria           

## 2.3 Identificar Outliers

In [28]:
df.describe()

,unidades_vendidas,importe_sin_iva,ranking_pedido_DESC,num_paso_rutina,size,fecha_albaran
count,3240516.0,3.240516e+06,3240516.0,3240516.0,3.240516e+06,3240516
mean,1.171035,1.235089e+01,19747.737377,3.537343,7.000329e+01,2024-07-19 21:11:41.075015168
min,-1.0,0.000000e+00,1.0,0.0,0.000000e+00,2024-01-02 00:00:00
25%,1.0,3.920000e+00,1.0,3.0,3.000000e+01,2024-05-08 00:00:00
50%,1.0,1.236000e+01,2.0,3.0,5.000000e+01,2024-07-05 00:00:00
75%,1.0,1.768000e+01,6.0,5.0,1.000000e+02,2024-11-11 00:00:00
max,10000.0,1.782720e+04,263577.0,9.0,5.000000e+02,2025-04-29 00:00:00
std,10.870277,3.392346e+01,59276.970248,1.73664,7.133592e+01,NaN


A continuación se muestran las estadísticas descriptivas de las principales variables cuantitativas del dataset:

- **`unidades_vendidas`**:
  - Se detectan valores negativos (`min = -1`), que **podrían ser devoluciones**.
  - El valor máximo es de **10.000 unidades**, muy por encima de la mediana (1), lo que sugiere **posibles pedidos atípicos o errores de carga**.

- **`importe_sin_iva`**:
  - Importes mínimos de **0 €** (regalos o errores).
  - El valor máximo es de **17.827,20 €**, mientras que el 75% de los pedidos no superan los **17,74 €**.  
    Esto indica que existen **valores extremos que pueden distorsionar el análisis económico**.

- **`size`**:
  - Valores entre **0 y 500**, con una media de **69,9** y una desviación estándar elevada (71,3).  
    Los valores extremos podrían deberse a envases de 500ml de los productos.

---
Acciones a realizar:
> Corregir el campo **`importe_sin_iva`**, dividiendo el valor total entre el número de unidades vendidas para obtener el precio unitario real en una columna nueva (**`precio_unitario`**).

In [29]:
df['precio_unitario'] = df['importe_sin_iva'] / df['unidades_vendidas']

In [30]:
df['precio_unitario'].head(5)

,precio_unitario
0,0.0
1,25.67
2,0.0
3,29.68
4,22.39


### 2.3.1 Poner precio a gifting

Ahora que tenemos una columna con el precio unitario real de cada `sku`
 puedo aplicar el precio de los productos regalados

In [31]:
# Creamos un diccionario con la mediana del precio unitario por sku (solo de sku vendidos)
sku_preus = df[df['gifting'] == False].groupby('sku')['precio_unitario'].median().to_dict()

# Susituimos los precios de los productos regalados
df.loc[(df['gifting'] == True) & (df['precio_unitario'] == 0), 'precio_unitario'] = \
    df.loc[(df['gifting'] == True) & (df['precio_unitario'] == 0), 'sku'].map(sku_preus)

In [32]:
df.describe()

,unidades_vendidas,importe_sin_iva,ranking_pedido_DESC,num_paso_rutina,size,fecha_albaran,precio_unitario
count,3240516.0,3.240516e+06,3240516.0,3240516.0,3.240516e+06,3240516,3190562.0
mean,1.171035,1.235089e+01,19747.737377,3.537343,7.000329e+01,2024-07-19 21:11:41.075015168,13.234431
min,-1.0,0.000000e+00,1.0,0.0,0.000000e+00,2024-01-02 00:00:00,0.0
25%,1.0,3.920000e+00,1.0,3.0,3.000000e+01,2024-05-08 00:00:00,9.25
50%,1.0,1.236000e+01,2.0,3.0,5.000000e+01,2024-07-05 00:00:00,12.36
75%,1.0,1.768000e+01,6.0,5.0,1.000000e+02,2024-11-11 00:00:00,17.24
max,10000.0,1.782720e+04,263577.0,9.0,5.000000e+02,2025-04-29 00:00:00,71.39
std,10.870277,3.392346e+01,59276.970248,1.73664,7.133592e+01,NaN,5.942538


# 3. Exportar datos limpios

In [33]:
client = bigquery.Client()

# --------------------------------
# Exporta el DataFrame limpio principal
# --------------------------------
table_id = 'freshly-case-study-468308.freshly_data.df_cleaned'

job_config = bigquery.LoadJobConfig(write_disposition="WRITE_TRUNCATE")

job = client.load_table_from_dataframe(df, table_id, job_config=job_config)
job.result()


# --------------------------------
# Exporta el resumen de los 'concerns'
# --------------------------------
concerns_table_id = 'freshly-case-study-468308.freshly_data.concerns_summary'

concerns_job_config = bigquery.LoadJobConfig(write_disposition="WRITE_TRUNCATE")

concerns_job = client.load_table_from_dataframe(df_concerns, concerns_table_id, job_config=concerns_job_config)
concerns_job.result()

print("Tabla exportada correctamente a BigQuery.")

Tabla exportada correctamente a BigQuery.
